## Trained Vowel Classfication From Speech
- created: 13 Apr 2020 </br>
To detect the error in an inversion data because the formant cannot be used to compare the model

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
import tensorflow as tf
from tensorflow import keras
from joblib import dump, load
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from time import time
from datetime import datetime
import shutil 
import argparse
from functools import partial
import pickle
from IPython.display import Audio
from sklearn.metrics import average_precision_score, precision_score, recall_score, roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Reshape, GRU, InputLayer, AlphaDropout, Activation, BatchNormalization, Dropout, Flatten, Dense, Bidirectional, LSTM, Conv1D, SpatialDropout1D, Concatenate, Multiply, Add

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
tf.random.set_seed(42)

## Function

In [3]:
def prep_data(data_dir,prep_folder):

    features = np.load(join(data_dir,prep_folder,'features.npy'))
    
    with open(join(data_dir,'syllable_name.txt')) as f:
        label = np.array([char for line in f for word in line.split(',') for char in word.strip()])
        
    print('Predict Data {} {}'.format(features.shape, label.shape))
    return features, label

In [4]:
def evaluate_model(y_pred, label):
    y_pred = enc.inverse_transform(y_pred)
    label = enc.inverse_transform(label)
#     print('Confusion Matrix')
    print(confusion_matrix(label, y_pred))
    print('Classification Report')
    print(classification_report(label, y_pred))

## Load Data 

In [5]:
DATA_PATH = '../../data/d_records/record_total_data/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)

Predict Data (3078, 23, 39) (3078,)


In [6]:
set(label)

{'7', 'E', 'M', 'O', 'a', 'e', 'i', 'o', 'u'}

In [7]:
label.shape

(3078,)

### One-hot label 

In [8]:
label=label.reshape(-1,1)

enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(label)

label_oe = enc.transform(label).toarray()

# dump(enc, 'onehot_pred.joblib')

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label_oe, test_size=0.2)

### Defined (Between) Embedding Model

In [10]:
PHONETIC_OUTPUT = len(['7', 'E', 'M', 'O', 'a', 'e', 'i', 'o', 'u'])
EMBEDDED_FEATURE = 64

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)

def embeded_model(input_shape_1,input_shape_2):
    inputs = keras.Input(shape=(input_shape_1,input_shape_2))
    x = pLSTM(EMBEDDED_FEATURE)(inputs)
    x = Dropout(rate=0.5)(x)
    x = pLSTM(EMBEDDED_FEATURE, return_sequences=False)(x)
    x = Dropout(rate=0.5)(x)
    outputs = Dense(PHONETIC_OUTPUT, activation='softmax', kernel_initializer='he_uniform')(x)
    return keras.Model(inputs=inputs, outputs=outputs)

In [11]:
embedded = embeded_model(23, 39)
embedded.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23, 39)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 23, 64)            26624     
_________________________________________________________________
dropout (Dropout)            (None, 23, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 9)                 585       
Total params: 60,233
Trainable params: 60,233
Non-trainable params: 0
_________________________________________________________

### Compile Model 

In [12]:
embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [22]:
PATIENCE = 15
BATCH_SIZE = 128
EPOCHS = 200

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=0, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

### Train Model 

In [35]:
from sklearn.metrics import roc_auc_score
def permutation_test(X_train, y_train, X_test, y_test, nsamples=10):
    idx1 = np.arange(X_train.shape[0])
    idx2 = np.arange(X_test.shape[0])
    auc_values = np.empty(nsamples)
    for b in range(nsamples):
        np.random.shuffle(idx1)  # Shuffles in-place
        np.random.shuffle(idx2)
        embedded = embeded_model(23, 39)
        embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
        embedded.fit(X_train, y_train[idx1], batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=0, validation_split=0.2, callbacks=callback_list)
        pred = embedded.predict(X_test)
#         print(pred)
        roc_auc = roc_auc_score(y_test[idx2].ravel(), pred.ravel())
        auc_values[b] = roc_auc
        print('Round {}'.format(b))
    embedded = embeded_model(23, 39)
    embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    embedded.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=0, validation_split=0.2, callbacks=callback_list)
    pred = embedded.predict(X_test)
    roc_auc = roc_auc_score(y_test.ravel(), pred.ravel())
    return roc_auc, np.mean(auc_values >= roc_auc)

In [36]:
auc, mean = permutation_test(X_train, y_train, X_test, y_test, nsamples=10)

Round 0
Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9


In [39]:
auc

0.9975454993464328

In [40]:
mean

0.0

In [41]:
def bootstrap_auc( X_train, y_train, X_test, y_test, nsamples=1000):
    auc_values = []
    for b in range(nsamples):
        idx = np.random.randint(X_train.shape[0], size=X_train.shape[0])
        embedded = embeded_model(23, 39)
        embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
        embedded.fit(X_train[idx], y_train[idx], batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=0, validation_split=0.2, callbacks=callback_list)
        pred = embedded.predict(X_test)
        roc_auc = roc_auc_score(y_test.ravel(), pred.ravel())
        auc_values.append(roc_auc)
        print('Round {}'.format(b))
    return np.percentile(auc_values, (2.5, 97.5))

In [42]:
boost_result = bootstrap_auc(X_train, y_train, X_test, y_test, nsamples=10)

In [43]:
boost_result

array([0.98647867, 0.99530752])

In [151]:
# embedded.save('pred_vowel.hdf5')

In [153]:
pred = embedded.predict(X_test)
evaluate_model(pred, y_test)

[[71  4  1  1  0  2  0  0  0]
 [ 1 70  0  0  0  1  0  0  0]
 [ 0  0 58  0  0  2  1  0  0]
 [ 0  0  0 73  0  0  0  1  0]
 [ 0  0  0  1 53  0  0  0  0]
 [ 0  0  0  0  0 64  3  0  0]
 [ 0  0  0  0  0  1 66  0  0]
 [ 0  0  0  0  0  0  0 64  1]
 [ 0  0  0  0  0  0  1  2 74]]
Classification Report
              precision    recall  f1-score   support

           7       0.99      0.90      0.94        79
           E       0.95      0.97      0.96        72
           M       0.98      0.95      0.97        61
           O       0.97      0.99      0.98        74
           a       1.00      0.98      0.99        54
           e       0.91      0.96      0.93        67
           i       0.93      0.99      0.96        67
           o       0.96      0.98      0.97        65
           u       0.99      0.96      0.97        77

    accuracy                           0.96       616
   macro avg       0.96      0.96      0.96       616
weighted avg       0.96      0.96      0.96       616



### Train using all data 

In [ ]:
embedded = embeded_model(23, 39)
embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
history = embedded.fit(features,label_oe,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    callbacks=callback_list)

In [155]:
embedded.save('pred_vowel_all_data.hdf5')

## Load Model

In [20]:
embedded = models.load_model('pred_vowel_all_data.hdf5')
enc = load('onehot_pred.joblib')

## Predict 31
- BiLSTM 

In [21]:
DATA_PATH = '../../data/inversion/inversion_31/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

Predict Data (1944, 23, 39) (1944,)
[[177  16  16   0   0   5   1   1   0]
 [  6 196   1   0   5   8   0   0   0]
 [  2   1 164   0   0  29  18   0   2]
 [  2   6   2 163  29   0   0  14   0]
 [  1  10   4  12 188   1   0   0   0]
 [  0   2   2   0   0 211   1   0   0]
 [  0   0   6   0   0  10 197   0   3]
 [ 10   2   1   7   0   5   0 145  46]
 [  4   2  20   0   0   5  15  12 158]]
Classification Report
              precision    recall  f1-score   support

           7       0.88      0.82      0.85       216
           E       0.83      0.91      0.87       216
           M       0.76      0.76      0.76       216
           O       0.90      0.75      0.82       216
           a       0.85      0.87      0.86       216
           e       0.77      0.98      0.86       216
           i       0.85      0.91      0.88       216
           o       0.84      0.67      0.75       216
           u       0.76      0.73      0.74       216

    accuracy                           0.82     

In [22]:
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)
precision = precision_score(class_actual, class_pred, average='macro')
recall = recall_score(class_actual, class_pred, average='macro')
auc = roc_auc_score(label_oe, pred)
print('Precision: {} Recall: {} AUC: {}'.format(precision,recall,auc))

Precision: 0.8255784132730615 Recall: 0.8225308641975309 AUC: 0.951728252171925
